### Atividade Prática 1: classificação

#### Brazilian Symposium on Bioinformatics (BSB) 2024

Copyright (C) 2021 Marcelo S. Reis.

<a href="mailto:marcelo.reis@butantan.gov.br">marcelo.reis@butantan.gov.br</a>

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program. If not, see http://www.gnu.org/licenses/.

Hoje vamos implementar algoritmos de aprendizagem não-supervisionada para realizar dois agrupamentos, um pelo k-médias (*k-means*) e o outro por aglomeração hierárquica (*hierarchical agglomerative clustering*).


### Sumário <a class="anchor" id="topo"></a>

* [Parte 1: Resolvendo dependências](#parte_01).
* [Parte 2: Carregando o conjunto de dados "breast cancer"](#parte_02).
* [Parte 3: Executando regressão linear em duas variáveis de "breast cancer"](#parte_03).
* [Parte 4: Avaliando o desempenho da regressão linear](#parte_04).
* [Parte 5: Classificando "breast cancer" com todas as variáveis](#parte_05)

### Parte 1: Resolvendo dependências <a class="anchor" id="parte_01"></a>

Aqui carregamos bibliotecas e métodos que serão utilizados ao longo deste caderno.




In [ ]:
# !pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# !pip install numpy
import numpy as np

# !pip install pandas
import pandas as pd

# !pip install seaborn
import seaborn as sn

# !pip install sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn import utils

from sklearn.linear_model import LinearRegression


[Voltar ao sumário.](#topo)

### Parte 2: Carregando o conjunto de dados "breast cancer"

[Breast Cancer Wisconsin (Diagnostic) Dataset](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)


Conjunto de dados clássico de classificação binária (2 classes), com uma amostra de 569 pontos (212 malignos, 357 benignos) e 30 características.


<a class="anchor" id="parte_02"></a>

In [ ]:
data = datasets.load_breast_cancer()

print(data.feature_names, end='\n\n')
print(data.target_names)


X = data.data
y = data.target

y = np.where(y == 0, -1, y)
X, y = utils.shuffle(X, y, random_state=42)

scaler = preprocessing.StandardScaler()

# Vamos utilizar 300 pontos para treinamento e 269 para teste
#
scaler.fit(X[1:301])
X = scaler.transform(X)

[Voltar ao sumário.](#topo)

### Parte 3: Executando regressão linear em duas variáveis de "breast cancer" <a class="anchor" id="parte_03"></a>

A equação de um modelo de regressão linear simples é:

$$
y = \beta_0 + \beta_1 x + \epsilon
$$

Onde:
- \( y \): Variável dependente (resposta).
- \( x \): Variável independente (preditor).
- $\beta_0$: Intercepto (valor de \( y \) quando \( x = 0 \)).
- $\beta_1$: Inclinação (taxa de mudança em \( y \) para uma mudança unitária em \( x \)).
- $\epsilon$: Termo de erro (captura o ruído ou a variabilidade não explicada pelo modelo).

Para regressão linear múltipla com \( n \) preditores, a fórmula é:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_n x_n + \epsilon
$$

Ou, em forma de matriz:

$$
\mathbf{y} = \mathbf{X} \boldsymbol{\beta} + \boldsymbol{\epsilon}
$$

Onde:
- $\mathbf{y}$: Vetor de variáveis ​​dependentes.
- $\mathbf{X}$: Matriz de variáveis ​​independentes (incluindo uma coluna de uns para o intercepto).
- $\boldsymbol{\beta}$: Vetor de coeficientes.
- $\boldsymbol{\epsilon}$: Vetor de erros.

Agora vamos executar o algoritmo de regressão linear sobre duas variáveis de nosso dataset. Vamos escolher as duas primeiras variáveis: raio médio e textura média do tumor.

In [ ]:
plt.figure(figsize=(8, 8))
plt.scatter(X[0:300, 0], X[0:300, 1], c=y[0:300])
plt.xlabel('Mean radius')
plt.ylabel('Mean texture')
plt.title('Conunto de treinamento')
plt.show()

In [ ]:
model  = LinearRegression().fit(X[0:300, 0:2], y[0:300])
y_pred = model.predict(X[:, 0:2])
y_pred = np.where(y_pred < 0, -1, 1)


plt.figure(figsize=(20, 10))

a = -model.coef_[0] / model.coef_[1]
b = -model.intercept_ / model.coef_[1]

plt.subplot(121)
plt.scatter(X[0:300, 0], X[0:300, 1], c=y_pred[0:300])
plt.xlabel('Mean radius')
plt.ylabel('Mean texture')
plt.title('Conjunto de treinamento')

plt.plot(np.linspace(-2, 4, 10), a * np.linspace(-2, 4, 10) + b, c='blue')
plt.ylim(-3, 6)



plt.subplot(122)
plt.scatter(X[300:570, 0], X[300:570, 1], c=y_pred[300:])
plt.xlabel('Mean radius')
plt.ylabel('Mean texture')
plt.title('Conjunto de teste')

plt.plot(np.linspace(-2, 4, 10), a * np.linspace(-2, 4, 10) + b, c='blue')
plt.ylim(-3, 6)


plt.show()

[Voltar ao sumário.](#topo)

### Parte 4: Avaliando o desempenho da regressão linear <a class="anchor" id="parte_04"></a>

Para avaliar o desempenho da nossa regressão linear, utilizaremos matrizes de confusão, que é uma maneira eficiente de visualizar os acertos em termos de tipo de erro: falsos positivos (não é câncer mas nosso classificador diz que é) e falsos negativos (um câncer que o classificador deixa de assinalar como tal).

In [ ]:
data = { 'y': y[0:300], 'y_pred': y_pred[0:300] }
df = pd.DataFrame(data, columns=['y', 'y_pred'])

cm = pd.crosstab(df['y'], df['y_pred'], rownames=['classe real'], colnames=['classe prevista'])

plt.figure(figsize=(20, 10))
sn.heatmap(cm, annot=True)
plt.show()

In [ ]:
data = { 'y': y[300:], 'y_pred': y_pred[300:] }
df = pd.DataFrame(data, columns=['y', 'y_pred'])

cm = pd.crosstab(df['y'], df['y_pred'], rownames=['classe real'], colnames=['classe prevista'])

plt.figure(figsize=(20, 10))
sn.heatmap(cm, annot=True)
plt.show()

[Voltar ao sumário.](#topo)

### Parte 5: Classificando "breast cancer" com todas as variáveis <a class="anchor" id="parte_05"></a>
    
Podemos verificar se conseguimos melhorar a classificação acima repetindo a regressão linear utilizando desta vez todas as variáveis.

In [ ]:
model = LinearRegression().fit(X[0:300], y[0:300])

y_pred = model.predict(X)
y_pred = np.where(y_pred < 0, -1, 1)


data = { 'y': y[300:], 'y_pred': y_pred[300:] }
df = pd.DataFrame(data, columns=['y', 'y_pred'])

cm = pd.crosstab(df['y'], df['y_pred'], rownames=['classe real'], colnames=['classe prevista'])

plt.figure(figsize=(20, 10))
sn.heatmap(cm, annot=True)
plt.show()

[Voltar ao sumário.](#topo)

